In [ ]:
import pathlib
import sys
import urllib.request
from typing import Dict

import openmm
import openmm.app
from openff.units.openmm import from_openmm as from_openmm_quantity

from openff.interchange.drivers import get_openmm_energies
from openff.interchange.drivers.openmm import _get_openmm_energies
from openff.interchange.interop.openmm import from_openmm

sys.setrecursionlimit(5000)

In [ ]:
def openmm_pathway(
    pdb_file: openmm.app.PDBFile,
    force_field: openmm.app.ForceField,
) -> Dict[str, float]:
    """Evaluate the energy of this PDB file and force field using only OpenMM."""
    system = force_field.createSystem(pdb_file.topology)

    for idx, force in enumerate(system.getForces()):
        force.setForceGroup(idx)

    integrator = openmm.VerletIntegrator(1.0 * openmm.unit.femtoseconds)
    context = openmm.Context(system, integrator)

    context.setPeriodicBoxVectors(*pdb_file.topology.getPeriodicBoxVectors())
    context.setPositions(pdb_file.getPositions())

    return {
        type(system.getForce(index)): context.getState(
            getEnergy=True, groups={index}
        ).getPotentialEnergy()
        for index in range(system.getNumForces())
    }

In [ ]:
if not pathlib.Path("protein.pdb").is_file():
    urllib.request.urlretrieve(
        "https://raw.githubusercontent.com/bayer-science-for-a-better-life/abfe-benchmark/a3b15632d2f419857e2ba73a6922de6f09a66caa/structures/brd4/protein.pdb",
        "brd4.pdb",
    )

protein = openmm.app.PDBFile("brd4.pdb")
amber_force_fields = openmm.app.ForceField("amber14-all.xml", "amber14/tip3pfb.xml")

amber_force_fields.createSystem(protein.topology)

In [ ]:
protein.topology.getPeriodicBoxVectors()

In [ ]:
converted_interchange = from_openmm(
    topology=protein.topology,
    system=amber_force_fields.createSystem(protein.topology),
    positions=from_openmm_quantity(protein.getPositions()),
    box_vectors=from_openmm_quantity(protein.topology.getPeriodicBoxVectors()),
)

As a quick sanity check, get the single-point energy of this configuration.

In [ ]:
print(get_openmm_energies(converted_interchange))

In [ ]:
openmm_pathway(protein, amber_force_fields)